In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, PrecisionRecallDisplay, precision_recall_curve

In [ ]:
thresholds = {
    "VGG-Face": {"cosine": 0.40, "euclidean": 0.60, "euclidean_l2": 0.86}, # 0.86
    "Facenet": {"cosine": 0.40, "euclidean": 10, "euclidean_l2": 1.04}, # l2 0.80
    "Facenet512": {"cosine": 0.30, "euclidean": 23.56, "euclidean_l2": 1.04},
    "ArcFace": {"cosine": 0.68, "euclidean": 4.15, "euclidean_l2": 1.15}  # 1.13
}

In [ ]:
def make_predictions(threshold, res_df):
    res_df["prediction"] = "No" #init
    idx = res_df[res_df.distance <= threshold].index
    res_df.loc[idx, 'prediction'] = 'Yes'
    return res_df

In [ ]:
def calculate_metrics(results_data_frame):
    cm = confusion_matrix(results_data_frame.decision.values, results_data_frame.prediction.values)
    tn, fp, fn, tp = cm.ravel()
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn)/(tn + fp +  fn + tp)
    f1 = 2 * (precision * recall) / (precision + recall)
    # print(cm)
    # cm_display = ConfusionMatrixDisplay(cm).plot()
    # print("precision ", precision)
    # print("recall ", recall)
    # print("accuracy ", accuracy)
    # prec, recall, _ = precision_recall_curve(results_data_frame.decision.values, results_data_frame.prediction.values, pos_label="Yes")
    # pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()
    return [precision, recall, accuracy, f1]

In [ ]:
for selected_model in thresholds.keys():
    # selected_model = "Facenet512"
    threshold = thresholds.get(selected_model).get("euclidean_l2")
    df = pd.read_csv(f"results/result_{selected_model}.csv")
    # tp_mean = round(df[df.decision == "Yes"].distance.mean(), 4)
    # tp_std = round(df[df.decision == "Yes"].distance.std(), 4)
    # fp_mean = round(df[df.decision == "No"].distance.mean(), 4)
    # fp_std = round(df[df.decision == "No"].distance.std(), 4)
    # print("positive: ", tp_mean, tp_std)
    # print("negative: ", fp_mean, fp_std)
    # sigma = 1
    # threshold = round(tp_mean + sigma * tp_std, 4)
    # print("statistical threshold, ", sigma, " sigma ", threshold)

    plt.figure()
    df[df.decision == "Yes"].distance.plot.kde()
    df[df.decision == "No"].distance.plot.kde()
    plt.legend(["Positive pairs", "Negative pairs"])
    plt.title(f'Funkcja gęstości prawdopodobnieństwa dla modelu {selected_model}')
    plt.xlabel("Odleglość pomiędzy wektorami cech")
    plt.ylabel("Gęstość prawdopodobieństwa")
    plt.xlim(left=0)
    plt.grid()
    # plt.axhline(0,color='red')
    # plt.axvline(0,color='red')
    # plt.axvline(threshold, color="green")
    plt.show()
    df = make_predictions(threshold, df)
    calculate_metrics(df)

In [ ]:
noise_type = "gaussian"
x_axis = np.arange(0, 0.3, 0.05).tolist()

results = {}

for model in thresholds.keys():
    threshold = thresholds.get(model).get("euclidean_l2")
    result = []
    for noise in x_axis:
        if noise == 0: 
            df = pd.read_csv(f"results/result_{model}.csv")   
        else:
            num = f"{noise:.2f}".replace(".", "_")
            df = pd.read_csv(f"results/result_{model}_{noise_type}_{num}.csv")  
        df = make_predictions(threshold, df)
        precision, recall, accuracy, f1 = calculate_metrics(df)
        print(precision, recall, accuracy)
        result.append([precision, recall, accuracy, f1])
    results[model] = result

plt.figure(1)
plt.figure(2)
plt.figure(3)
models = []
for model in thresholds.keys():
    models.append(model)
    y_axis = [i[0] for i in results[model]]
    # print(x_axis, y_axis)
    
    plt.figure(1)
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Precyzja dla szumów Gaussa")
    plt.ylabel("Precyzja [%]")
    plt.xlabel("Odchylenie standardowe dla szumu Gaussa")

    plt.figure(2)
    y_axis = [i[1] for i in results[model]]
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Czułość dla szumów Gaussa")
    plt.ylabel("Czułość [%]")
    plt.xlabel("Odchylenie standardowe dla szumu Gaussa")

    plt.figure(3)
    y_axis = [i[2] for i in results[model]]
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Dokładność dla szumów Gaussa")
    plt.ylabel("Dokładność [%]")
    plt.xlabel("Odchylenie standardowe dla szumu Gaussa")

plt.figure(1)
plt.legend()
plt.figure(2)
plt.legend()
plt.figure(3)  
plt.legend()

In [ ]:
noise_type = "adj_gamma"
x_axis = np.arange(0, 10, 2).tolist()

results = {}

for model in thresholds.keys():
    threshold = thresholds.get(model).get("euclidean_l2")
    result = []
    for noise in x_axis:
        if noise == 0: 
            df = pd.read_csv(f"results/result_{model}.csv")   
        else:
            num = f"{noise:.2f}".replace(".", "_")
            df = pd.read_csv(f"results/result_{model}_{noise_type}_{num}.csv")  
        df = make_predictions(threshold, df)
        precision, recall, accuracy, f1 = calculate_metrics(df)
        print(precision, recall, accuracy)
        result.append([precision, recall, accuracy, f1])
    results[model] = result

plt.figure(1)
plt.figure(2)
plt.figure(3)
models = []
for model in thresholds.keys():
    models.append(model)
    y_axis = [i[0] for i in results[model]]
    # print(x_axis, y_axis)
    
    plt.figure(1)
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Precyzja po korekcji gamma")
    plt.ylabel("Precyzja [%]")
    plt.xlabel("Wartość parametru gamma")
    plt.xscale("log")

    plt.figure(2)
    y_axis = [i[1] for i in results[model]]
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Czułość po korekcji gamma")
    plt.ylabel("Czułość [%]")
    plt.xlabel("Wartość parametru gamma")

    plt.figure(3)
    y_axis = [i[2] for i in results[model]]
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Dokładność po korekcji gamma")
    plt.ylabel("Dokładność [%]")
    plt.xlabel("Wartość parametru gamma")

plt.figure(1)
plt.legend()
plt.figure(2)
plt.legend()
plt.figure(3)  
plt.legend()

In [ ]:
noise_type = "gaussian_blur"
x_axis = np.arange(0, 3, 0.5).tolist()

results = {}

for model in thresholds.keys():
    threshold = thresholds.get(model).get("euclidean_l2")
    result = []
    for noise in x_axis:
        if noise == 0: 
            df = pd.read_csv(f"results/result_{model}.csv")   
        else:
            num = f"{noise:.2f}".replace(".", "_")
            df = pd.read_csv(f"results/result_{model}_{noise_type}_{num}.csv")  
        df = make_predictions(threshold, df)
        precision, recall, accuracy, f1 = calculate_metrics(df)
        print(precision, recall, accuracy)
        result.append([precision, recall, accuracy, f1])
    results[model] = result

plt.figure(1)
plt.figure(2)
plt.figure(3)
models = []
for model in thresholds.keys():
    models.append(model)
    y_axis = [i[0] for i in results[model]]
    # print(x_axis, y_axis)
    
    plt.figure(1)
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Precyzja dla rozmycia Gaussa")
    plt.ylabel("Precyzja [%]")
    plt.xlabel("Odchylenie standardowe dla rozmycia Gaussa")
    plt.xscale("log")

    plt.figure(2)
    y_axis = [i[1] for i in results[model]]
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Czułość dla rozmycia Gaussa")
    plt.ylabel("Czułość [%]")
    plt.xlabel("Odchylenie standardowe dla rozmycia Gaussa")

    plt.figure(3)
    y_axis = [i[2] for i in results[model]]
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Dokładność dla rozmycia Gaussa")
    plt.ylabel("Dokładność [%]")
    plt.xlabel("Odchylenie standardowe dla rozmycia Gaussa")

plt.figure(1)
plt.legend()
plt.figure(2)
plt.legend()
plt.figure(3)  
plt.legend()

In [ ]:
noise_type = "s_v_"
x_axis = np.arange(0, 0.1, 0.02).tolist()

results = {}

for model in thresholds.keys():
    if model in ["ArcFace", "Facenet512"]:
        continue
    threshold = thresholds.get(model).get("euclidean_l2")
    result = []
    for noise in x_axis:
        if noise == 0: 
            df = pd.read_csv(f"results/result_{model}.csv")   
        else:
            num = f"{noise:.2f}".replace(".", "_")
            df = pd.read_csv(f"results/result_{model}_{noise_type}_{num}.csv")  
        df = make_predictions(threshold, df)
        precision, recall, accuracy, f1 = calculate_metrics(df)
        print(precision, recall, accuracy)
        result.append([precision, recall, accuracy, f1])
    results[model] = result

plt.figure(1)
plt.figure(2)
plt.figure(3)
models = []
for model in thresholds.keys():
    models.append(model)
    y_axis = [i[0] for i in results[model]]
    # print(x_axis, y_axis)
    
    plt.figure(1)
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Precyzja dla szumów salt and pepper")
    plt.ylabel("Precyzja [%]")
    plt.xlabel("Ilość pikseli zmienionych")
    plt.xscale("log")

    plt.figure(2)
    y_axis = [i[1] for i in results[model]]
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Czułość dla salt and pepper")
    plt.ylabel("Czułość [%]")
    plt.xlabel("Ilość pikseli zmienionych")

    plt.figure(3)
    y_axis = [i[2] for i in results[model]]
    plt.plot(x_axis, y_axis, ".-", label=model)
    plt.title("Dokładność dla salt and pepper")
    plt.ylabel("Dokładność [%]")
    plt.xlabel("Ilość pikseli zmienionych")

plt.figure(1)
plt.legend()
plt.figure(2)
plt.legend()
plt.figure(3)  
plt.legend()